### Capstone Project - The Battle of Neighborhoods

## 1. INTRODUCTION
In metropolitan areas, one of the main means of transportation is the metro. In London, this is no different - besides the fact the locals rather call it 'Underground'.
As the streets are packed with cars, people tend to prefer to live within a walking distance from metro stations. 
In order to provide more information for people seeking for new homes in London, this work attempts to provide more information about the areas around metro stations in the city, more specifically about leisure options.
#### Problem: 
Group metro stations of London according to the leisure options near them.
#### Target audience: 
People looking for areas near London Underground stations to live, basing their choices on the leisure options available.
#### Data sources:
- The *London metro location* will be sourced from the table at https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations#Tube_Stations_List , which contains geolocation info (lat and long) of all stations in the city.
- *Leisure option data* will be retrieved using Foursquare API, which provides geo information and a short description about locations.

## 2. DATA

In [1]:
#London Underground data
import pandas as pd
underground_df = pd.read_html('https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations#Tube_Stations_List', header = 0)
underground_df = underground_df[0]
underground_df = underground_df[['Name', 'Latitude', 'Longitude']]
underground_df.rename(columns={'Name':'Underground',
                          'Latitude':'Latitude',
                          'Longitude':'Longitude'
                              }, 
                 inplace=True)

In [2]:
#we need to transform the Latitude and Longitude of the case below to Numeric
underground_df[underground_df.Underground == 'Canada Water']

,Underground,Latitude,Longitude
42,Canada Water,51.49787 +/- 0.000011 (from 3 readings),-0.04967 +/- 0.000008 (from 3 readings)


In [3]:
#deleting the case
underground_df= underground_df[underground_df.Underground != 'Canada Water']

In [4]:
#some stations have two exits, and both have coordinates in the dataframe. 
#As this will not imapct our analysis, we drop the repeated entries from the dataframe
underground_df = underground_df.drop_duplicates(['Underground'])

In [5]:
#Getting the shape of the Underground Dataframe
underground_df.shape

(290, 3)

## 3. METHODOLOGY

In [6]:
#Importing libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.4893335, -0.144055084527687.


In [8]:
#Creating a map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

df = underground_df #to make it easier to use the code below provided in the course

# add markers to map
for lat, lng, underground in zip(df['Latitude'], df['Longitude'], df['Underground']):
    label = '{}, {}'.format(df, underground)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        fill_color='#3186cc',
        parse_html=False).add_to(map_london)  
    
map_london

In [9]:
# @hidden_cell
#Connecting to Foursquare IP
CLIENT_ID = 'A21LAIM03FSYEL0XAPAQMKSFG2A0HO3X5IZ0HQYHCO2BTAJK' # your Foursquare ID
CLIENT_SECRET = '44RZOJOOMKCKMQ3JTSEYAIKUDASWPSHDPVFY1GOQUX2RWJNC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [10]:
#Lat and Long of the First Underground in the dataframe 
underground_latitude = underground_df.loc[0, 'Latitude'] # underground latitude value
underground_longitude = underground_df.loc[0, 'Longitude'] # underground longitude value

underground_name = underground_df.loc[0, 'Underground'] # underground name

print('Latitude and longitude values of {} are {}, {}.'.format(underground_name, 
                                                               underground_latitude, 
                                                               underground_longitude))

Latitude and longitude values of Acton Town are 51.502500, -0.278126.


In [11]:
#getting the foursquare url
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    underground_latitude, 
    underground_longitude, 
    radius, 
    LIMIT)

In [12]:
results = requests.get(url).json()

In [13]:
# setting function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Apple Tree Cakes,Coffee Shop,51.503174,-0.280474
1,WP Fish X Chips,Fish & Chips Shop,51.502580,-0.281240
2,London Transport Museum Depot,Museum,51.504175,-0.280622
3,Jungles Cafe,Breakfast Spot,51.503272,-0.280173
4,M&S Simply Food,Grocery Store,51.501229,-0.280420


In [15]:
#setting a function to get nearby venues and using it
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Underground', 
                  'Underground Latitude', 
                  'Underground Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
london_venues = getNearbyVenues(names=underground_df['Underground'],
                                   latitudes=underground_df['Latitude'],
                                   longitudes=underground_df['Longitude']
                                  )

Acton Town
Acton Central
Aldgate
Aldgate East
Alperton
Amersham
Angel
Archway
Arnos Grove
Arsenal
Baker Street
Balham
Bank
Barbican
Barking
Barkingside
Barons Court
Bayswater
Becontree
Belsize Park
Bermondsey
Bethnal Green
Blackfriars
Blackhorse Road
Bond Street
Borough
Boston Manor
Bounds Green
Bow Road
Brent Cross
Brixton
Bromley-by-Bow
Brondesbury
Brondesbury Park
Buckhurst Hill
Burnt Oak
Caledonian Road
Caledonian Road & Barnesbury
Camden Road
Camden Town
Canary Wharf
Cannon Street
Canonbury
Canons Park
Chalfont & Latimer
Chalk Farm
Chancery Lane
Charing Cross
Chesham
Chigwell
Chiswick Park
Chorleywood
City Thameslink
Clapham Common
Clapham North
Clapham South
Cockfosters
Colindale
Colliers Wood
Covent Garden
Croxley
Dagenham East
Dagenham Heathway
Dalston Kingsland
Debden
Dollis Hill
Ealing Broadway
Ealing Common
Earl's Court
East Acton
East Finchley
East Ham
East Putney
Eastcote
Edgware
Edgware Road (Bakerloo Line)
Edgware Road (Circle Line)
Elephant & Castle
Elm Park
Embankment


In [17]:
print(london_venues.shape)
london_venues.head()

(11741, 7)


,Underground,Underground Latitude,Underground Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton Town,51.502500,-0.278126,The Apple Tree Cakes,51.503174,-0.280474,Coffee Shop
1,Acton Town,51.502500,-0.278126,WP Fish X Chips,51.502580,-0.281240,Fish & Chips Shop
2,Acton Town,51.502500,-0.278126,London Transport Museum Depot,51.504175,-0.280622,Museum
3,Acton Town,51.502500,-0.278126,Jungles Cafe,51.503272,-0.280173,Breakfast Spot
4,Acton Town,51.502500,-0.278126,M&S Simply Food,51.501229,-0.280420,Grocery Store


In [18]:
london_venues.groupby('Underground').count()

,Underground Latitude,Underground Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Underground,,,,,,
Acton Central,17,17,17,17,17,17
Acton Town,10,10,10,10,10,10
Aldgate,100,100,100,100,100,100
Aldgate East,100,100,100,100,100,100
Alperton,7,7,7,7,7,7
Amersham,25,25,25,25,25,25
Angel,77,77,77,77,77,77
Archway,34,34,34,34,34,34
Arnos Grove,4,4,4,4,4,4


In [19]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 406 uniques categories.


In [88]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add Underground column back to dataframe
london_onehot['Underground'] = london_venues['Underground'] 

# move Underground column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_grouped = london_onehot.groupby('Underground').mean().reset_index()
london_grouped.head()

,Underground,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,College Science Building,Colombian Restaurant,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,

In [92]:
#deleting the "Metro Station" column, as it does not make sense to see if there are other metro stations around
london_grouped = london_grouped.drop(columns="Metro Station")

In [94]:
#each Underground along with the top 5 most common venues
num_top_venues = 5 # set to 5 because 10 seemed to be too large, and in many cases there was 0 frequency

for hood in london_grouped['Underground']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Underground'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acton Central----
                  venue  freq
0  Gym / Fitness Center  0.24
1                   Pub  0.18
2           Coffee Shop  0.12
3              Wine Bar  0.06
4                  Park  0.06


----Acton Town----
                venue  freq
0  Italian Restaurant   0.1
1    Kebab Restaurant   0.1
2       Grocery Store   0.1
3         Coffee Shop   0.1
4         Pizza Place   0.1


----Aldgate----
                  venue  freq
0                 Hotel  0.11
1           Coffee Shop  0.10
2          Cocktail Bar  0.05
3           Salad Place  0.04
4  Gym / Fitness Center  0.04


----Aldgate East----
               venue  freq
0        Coffee Shop  0.12
1                Pub  0.09
2              Hotel  0.08
3  Indian Restaurant  0.07
4               Café  0.06


----Alperton----
                       venue  freq
0                Supermarket  0.29
1             Sandwich Place  0.14
2       Gym / Fitness Center  0.14
3          Indian Restaurant  0.14
4  Middle Eastern Restaurant  0.

               venue  freq
0        Coffee Shop  0.12
1                Pub  0.08
2     Sandwich Place  0.05
3         Food Truck  0.04
4  French Restaurant  0.04


----Charing Cross----
          venue  freq
0         Hotel  0.05
1       Theater  0.04
2    Restaurant  0.04
3  Burger Joint  0.03
4      Wine Bar  0.03


----Chesham----
                  venue  freq
0           Coffee Shop  0.18
1           Supermarket  0.09
2                  Food  0.09
3  Other Great Outdoors  0.09
4             Bookstore  0.09


----Chigwell----
                           venue  freq
0          Performing Arts Venue  0.25
1                           Park  0.25
2                            Pub  0.25
3              Other Repair Shop  0.00
4  Paper / Office Supplies Store  0.00


----Chiswick Park----
                venue  freq
0                Café  0.08
1                 Pub  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.06
4           Gastropub  0.06


----Chorleywood----
               ve

                  venue  freq
0           Coffee Shop  0.13
1                  Café  0.08
2  Fast Food Restaurant  0.06
3        Soccer Stadium  0.06
4             Gastropub  0.04


----Gants Hill----
                     venue  freq
0              Pizza Place  0.13
1            Grocery Store  0.13
2                     Café  0.04
3  Health & Beauty Service  0.04
4       Chinese Restaurant  0.04


----Gloucester Road----
                venue  freq
0                Café  0.10
1               Hotel  0.06
2  Italian Restaurant  0.05
3             Exhibit  0.05
4              Bakery  0.04


----Golders Green----
                venue  freq
0         Coffee Shop  0.13
1   Korean Restaurant  0.10
2                Café  0.06
3  Turkish Restaurant  0.06
4         Supermarket  0.03


----Goldhawk Road----
                venue  freq
0               Hotel  0.15
1       Grocery Store  0.09
2                 Pub  0.06
3         Coffee Shop  0.06
4  Chinese Restaurant  0.06


----Goodge Street----

4     Coffee Shop  0.06


----Kilburn----
                 venue  freq
0        Grocery Store  0.08
1          Coffee Shop  0.08
2             Bus Stop  0.08
3  Fried Chicken Joint  0.04
4                 Café  0.04


----Kilburn Park----
                  venue  freq
0                   Pub  0.12
1         Grocery Store  0.09
2           Pizza Place  0.06
3                Bakery  0.06
4  Gym / Fitness Center  0.06


----King's Cross St.Pancras----
                venue  freq
0               Hotel  0.10
1         Coffee Shop  0.07
2                 Pub  0.05
3                Café  0.04
4  Italian Restaurant  0.03


----King's Cross Thameslink----
                venue  freq
0         Coffee Shop  0.08
1               Hotel  0.07
2                Café  0.05
3                 Pub  0.05
4  Italian Restaurant  0.03


----Kingsbury----
                  venue  freq
0         Grocery Store  0.21
1  Herbs & Spices Store  0.07
2     Indian Restaurant  0.07
3        Sandwich Place  0.07
4      

         venue  freq
0         Café  0.15
1  Coffee Shop  0.08
2  Golf Course  0.08
3        Hotel  0.08
4          Pub  0.08


----Piccadilly Circus----
                venue  freq
0        Cocktail Bar  0.05
1             Theater  0.05
2      Ice Cream Shop  0.05
3      Clothing Store  0.04
4  Seafood Restaurant  0.04


----Pimlico----
                venue  freq
0               Hotel  0.12
1                Park  0.09
2         Art Gallery  0.09
3              Garden  0.06
4  Italian Restaurant  0.06


----Pinner----
                venue  freq
0         Coffee Shop  0.12
1  Italian Restaurant  0.12
2                 Pub  0.08
3         Supermarket  0.08
4          Restaurant  0.08


----Plaistow----
            venue  freq
0     Rugby Pitch  0.25
1        Platform  0.12
2     Coffee Shop  0.12
3  Breakfast Spot  0.12
4             Pub  0.12


----Preston Road----
                  venue  freq
0         Grocery Store  0.15
1                   Bar  0.08
2     Indian Restaurant  0.08
3

               venue  freq
0              Hotel  0.25
1      Train Station  0.25
2      Grocery Store  0.25
3  Outdoor Sculpture  0.00
4               Park  0.00


----Sudbury Town----
                       venue  freq
0              Train Station  0.17
1                        Pub  0.17
2          Indian Restaurant  0.17
3  Middle Eastern Restaurant  0.17
4                   Bus Stop  0.17


----Surrey Quays----
               venue  freq
0               Café  0.10
1        Pizza Place  0.07
2      Train Station  0.05
3  Convenience Store  0.05
4           Bus Stop  0.05


----Swiss Cottage----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.08
2       Grocery Store  0.06
3                Café  0.04
4           Hotel Bar  0.04


----Temple----
          venue  freq
0       Theater  0.12
1           Pub  0.09
2         Hotel  0.06
3  Cocktail Bar  0.05
4   Coffee Shop  0.05


----Theydon Bois----
           venue  freq
0            Pub  0.33
1         

4  Chinese Restaurant   0.1


----Woodside Park----
                  venue  freq
0                  Café  0.50
1         Grocery Store  0.25
2          Soccer Field  0.25
3     Accessories Store  0.00
4  Outdoor Supply Store  0.00




In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Underground']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
underground_venues_sorted = pd.DataFrame(columns=columns)
underground_venues_sorted['Underground'] = london_grouped['Underground']

for ind in np.arange(london_grouped.shape[0]):
    underground_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

underground_venues_sorted.head()

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Acton Central,Gym / Fitness Center,Pub,Coffee Shop,Wine Bar,Bakery
1,Acton Town,Museum,Kebab Restaurant,Pizza Place,Coffee Shop,Fish & Chips Shop
2,Aldgate,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Salad Place
3,Aldgate East,Coffee Shop,Pub,Hotel,Indian Restaurant,Café
4,Alperton,Supermarket,Gym / Fitness Center,Indian Restaurant,Middle Eastern Restaurant,Asian Restaurant


In [97]:
# set number of clusters
kclusters = 7

london_grouped_clustering = london_grouped.drop('Underground', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 1, 5, 3, 1, 3, 4, 3])

In [98]:
# add clustering labels
underground_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = underground_df

# merge london_merged with london_data to add latitude/longitude for each Underground
london_merged = london_merged.join(underground_venues_sorted.set_index('Underground'), on='Underground')

london_merged.head() # check the last columns!

,Underground,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Acton Town,51.502500,-0.278126,3,Museum,Kebab Restaurant,Pizza Place,Coffee Shop,Fish & Chips Shop
1,Acton Central,51.50883531,-0.263033174,3,Gym / Fitness Center,Pub,Coffee Shop,Wine Bar,Bakery
3,Aldgate,51.51394,-0.07537,1,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Salad Place
4,Aldgate East,51.51514,-0.07178,1,Coffee Shop,Pub,Hotel,Indian Restaurant,Café
5,Alperton,51.54097,-0.30061,5,Supermarket,Gym / Fitness Center,Indian Restaurant,Middle Eastern Restaurant,Asian Restaurant


In [99]:
london_merged = london_merged.dropna() #dropping rows with NaN

In [108]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Underground'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[int(cluster)-1 ],
        fill=True,
        fill_color=rainbow[int(cluster) - 1 ],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3. ANALYSIS

In [101]:
#Cluster 0
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,Bayswater,Hotel,Pub,Coffee Shop,Café,Chinese Restaurant
30,Bow Road,Pub,Hotel,Coffee Shop,Burger Joint,Breakfast Spot
70,Ealing Common,Coffee Shop,Eastern European Restaurant,Hotel,Japanese Restaurant,Middle Eastern Restaurant
71,Earl's Court,Hotel,Café,Italian Restaurant,Pizza Place,Coffee Shop
97,Goldhawk Road,Hotel,Grocery Store,Pub,Coffee Shop,Chinese Restaurant
112,Hanger Lane,Fast Food Restaurant,Pub,Mediterranean Restaurant,Italian Restaurant,English Restaurant
116,Hatton Cross,Convenience Store,Hotel,Bus Station,Fast Food Restaurant,Gym / Fitness Center
133,Hyde Park Corner,Hotel,Lounge,Tea Room,Restaurant,Café
138,Kensington (Olympia),Hotel,Pub,Persian Restaurant,Indian Restaurant,Pizza Place
151,Lancaster Gate,Hotel,Café,Pub,Coffee Shop,Garden


In [102]:
#Cluster 1
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Aldgate,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Salad Place
4,Aldgate East,Coffee Shop,Pub,Hotel,Indian Restaurant,Café
7,Angel,Coffee Shop,Pub,Café,Vietnamese Restaurant,Korean Restaurant
11,Baker Street,Café,Coffee Shop,French Restaurant,Museum,Pub
14,Bank,Coffee Shop,Italian Restaurant,Restaurant,Hotel,Pub
15,Barbican,Food Truck,Coffee Shop,Hotel,Italian Restaurant,Café
24,Blackfriars,Coffee Shop,Italian Restaurant,Sandwich Place,Pub,Gym / Fitness Center
26,Bond Street,Coffee Shop,Clothing Store,Juice Bar,Café,Hotel
32,Brixton,Caribbean Restaurant,Coffee Shop,Pizza Place,Pub,Market
40,Camden Road,Pub,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Market


In [103]:
#Cluster 2
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,Barons Court,Pub,Hotel,Cocktail Bar,Garden,Tennis Court
33,Bromley-by-Bow,Pub,Historic Site,Café,Waterfront,Light Rail Station
36,Buckhurst Hill,Pub,Café,Italian Restaurant,Coffee Shop,Supermarket
45,Canonbury,Café,Pub,Coffee Shop,Italian Restaurant,Park
52,Chigwell,Park,Pub,Performing Arts Venue,Farm,Duty-free Shop
63,Croxley,Pub,Restaurant,Café,Grocery Store,Coffee Shop
99,Gospel Oak,Café,Convenience Store,Auto Garage,Pool,Playground
111,Hampstead Heath,Café,Pub,Bakery,Museum,Indian Restaurant
113,Harlesden,Middle Eastern Restaurant,Pub,Movie Theater,Train Station,Yoshoku Restaurant
123,Highgate,Pub,Café,Trail,Indian Restaurant,Japanese Restaurant


In [104]:
#Cluster 3
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Acton Town,Museum,Kebab Restaurant,Pizza Place,Coffee Shop,Fish & Chips Shop
1,Acton Central,Gym / Fitness Center,Pub,Coffee Shop,Wine Bar,Bakery
6,Amersham,Coffee Shop,Supermarket,Climbing Gym,Pizza Place,Grocery Store
8,Archway,Coffee Shop,Grocery Store,Pub,Pizza Place,Indian Restaurant
10,Arsenal,Pub,Gym / Fitness Center,Park,Sports Bar,Soccer Stadium
13,Balham,Coffee Shop,Pizza Place,Pub,Bakery,Indian Restaurant
16,Barking,Pub,Grocery Store,Pizza Place,Chinese Restaurant,Discount Store
17,Barkingside,Construction & Landscaping,Café,Supermarket,Soccer Field,Farm
21,Belsize Park,Café,Bakery,Pub,Gym / Fitness Center,Pizza Place
23,Bethnal Green,Pub,Coffee Shop,Café,Cocktail Bar,Pizza Place


In [105]:
#Cluster 4
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Arnos Grove,Grocery Store,Chinese Restaurant,Bus Stop,Park,Yoshoku Restaurant
20,Becontree,Construction & Landscaping,Women's Store,Grocery Store,Fast Food Restaurant,Yoshoku Restaurant
22,Bermondsey,Pub,Grocery Store,Coffee Shop,Indonesian Restaurant,Brazilian Restaurant
25,Blackhorse Road,Food & Drink Shop,Men's Store,Rock Club,Middle Eastern Restaurant,Dance Studio
28,Boston Manor,English Restaurant,Grocery Store,Canal Lock,Bus Stop,Yoshoku Restaurant
31,Brent Cross,Supermarket,Bus Stop,Grocery Store,Beer Garden,Coffee Shop
35,Brondesbury Park,Park,Pub,Grocery Store,Train Station,Deli / Bodega
37,Burnt Oak,Eastern European Restaurant,Coffee Shop,Bakery,Grocery Store,Ice Cream Shop
58,Clapham South,Grocery Store,Bar,Historic Site,Indian Restaurant,Fish Market
60,Colindale,Grocery Store,Convenience Store,Pizza Place,Park,Gym / Fitness Center


In [106]:
#Cluster 5
london_merged.loc[london_merged['Cluster Labels'] == 5, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Alperton,Supermarket,Gym / Fitness Center,Indian Restaurant,Middle Eastern Restaurant,Asian Restaurant
46,Canons Park,Indian Restaurant,Park,Bar,Farmers Market,Eastern European Restaurant
54,Chorleywood,Indian Restaurant,Deli / Bodega,Coffee Shop,Pub,Train Station
74,East Ham,Indian Restaurant,Discount Store,Bakery,Fast Food Restaurant,Gym / Fitness Center
114,Harrow & Wealdstone,Convenience Store,Gym / Fitness Center,Park,Sandwich Place,Gym
131,Hounslow East,Indian Restaurant,Bus Stop,Fast Food Restaurant,Jewelry Store,Asian Restaurant
132,Hounslow West,Indian Restaurant,Hotel,Fast Food Restaurant,Park,Grocery Store
179,North Wembley,Sports Club,Indian Restaurant,Rental Car Location,Train Station,Sandwich Place
185,Northwood Hills,Indian Restaurant,Convenience Store,Fast Food Restaurant,Grocery Store,Dessert Shop
189,Osterley,Indian Restaurant,Bus Stop,Hockey Field,Bus Station,Chinese Restaurant


In [107]:
#Cluster 6
london_merged.loc[london_merged['Cluster Labels'] == 6, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Underground,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
64,Dagenham East,Platform,Soccer Field,Gas Station,Restaurant,Farm
86,Fairlop,Gym / Fitness Center,Golf Course,Playground,Farmers Market,Park
152,Latimer Road,Gym / Fitness Center,Cosmetics Shop,Art Gallery,Modern European Restaurant,Park
165,Mill Hill East,Convenience Store,Supermarket,Park,Campground,Bakery
241,Stanmore,Platform,Park,Italian Restaurant,Supermarket,Pizza Place
263,Upminster,Platform,Coffee Shop,Bakery,Stationery Store,Greek Restaurant
282,West Ham,Platform,Fish & Chips Shop,Park,Trail,Café


### 3.1. Results

After some trial and error, it was found that setting 7 clusters was a good balance between the quality of information (how homogeneous the clusters are) and the amount of effort to analyze it and to take action upon (eventually, by increasing the number of clusters we would get too many groups to understand).

In doing so, the 7 clusters were well defined. They are described below:
- __Cluster 0__: Stations within this cluster are mainly surrounded by Hotels. Pubs are also important venues;
- __Cluster 1__: Around the stations in this cluster, there are a variety of venues with emphasis on Coffee Shops;
- __Cluster 2__: This cluster has Pubs and Cafés as the most common venues;
- __Cluster 3__: This is the largest cluster, meaning the most heterogeneous as well. Coffee Shops and Pubs are well offered around the stations here;
- __Cluster 4__: In this cluster, stations are surrounded by Grocery Stores;
- __Cluster 5__: If you are into Indian food, these stations are the place to live. Indian Restaurants are the most common venues here;
- __Cluster 6__: Gym and Fitness Center and Parks are well offered in this small cluster (only 7 stations).

In [ ]:
### 3.2. Discussion